In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os
import math



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [ ]:
raw_file="Raw data 26-April'25 According to 4_7.5.csv"
gst_file="GST 26-April'25 According to 4_7.5.csv"
tracker_file="EDD Tracker till 26th April'25.csv"
format_file="edd_format.csv"
ops_file="ops_status.csv"
old_tracker_file="old_tracker.csv"


In [ ]:
raw_data= pd.read_csv(raw_file,memory_map=True)
gst_data=pd.read_csv(gst_file,memory_map=True)
tracker_data=pd.read_csv(tracker_file,memory_map=True)
master_format=pd.read_csv(format_file,memory_map=True)
ops_status=pd.read_csv(ops_file,memory_map=True)
old_tracker=pd.read_csv(old_tracker_file,memory_map=True)

In [ ]:
raw_data.head(3)

In [ ]:
gst_data=gst_data.rename(columns={'_col3':'updated_at'})
gst_data.head(3)

In [ ]:
# Convert the two columns to datetime first
raw_data['mid_creation_date'] = pd.to_datetime(raw_data['mid_creation_date'], errors='coerce')
raw_data['channel_date'] = pd.to_datetime(raw_data['channel_date'], errors='coerce')

# Then format them to 'dd-mm-yyyy' string format
raw_data['mid_creation_date'] = raw_data['mid_creation_date'].dt.strftime('%d-%m-%Y')
raw_data['channel_date'] = raw_data['channel_date'].dt.strftime('%d-%m-%Y')

raw_data.head(3)

In [ ]:
tracker_data=tracker_data.fillna(0)
tracker_data.isna().sum()

In [ ]:
len(ops_status)+len(tracker_data)+len(old_tracker)

In [ ]:
tracker_data=tracker_data.rename(columns={'Bank Response':'bank_response','Final Remarks':'final_remarks'})
old_tracker=old_tracker.rename(columns={'bank_response':'bank_response','Final Remarks':'final_remarks'})
ops_status=ops_status.rename(columns={'sellerid':'bank_response','OPS status':'final_remarks'})



ops_status_data= pd.concat([ops_status, tracker_data,old_tracker], axis=0, ignore_index=True)
print('totol tracker data ',len(ops_status_data))
print(ops_status_data.isna().sum())


In [ ]:
len(raw_data)

In [ ]:
# Make sure everything is a string first
raw_data['sellerid'] = raw_data['sellerid'].astype(str)
raw_data['entity_id'] = raw_data['entity_id'].astype(str)
raw_data['parent_entity_id'] = raw_data['parent_entity_id'].astype(str)

# Calculate the length of each entity_id
raw_data['entity_len'] = raw_data['entity_id'].str.len()

# Extract from seller_id the last N characters based on entity_len
raw_data['seller_tail'] = raw_data.apply(lambda x: x['sellerid'][-int(x['entity_len']):], axis=1)


# Step 3: Compare both
raw_data['is_entity_match'] = raw_data['seller_tail'] == raw_data['entity_id']
raw_data['is_parent_match'] = raw_data['seller_tail'] == raw_data['parent_entity_id']

# Step 4: Create final match column
raw_data['final_match'] = raw_data['is_entity_match'] | raw_data['is_parent_match']

raw_data = raw_data[raw_data['final_match']]


len(raw_data)



In [ ]:
master_format[['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank']]=raw_data[['parent_id', 'mid','parent_entity_id','entity_id',
       'mid_creation_date', 'channel_date', 'cust_id', 'channel', 'edc_flag',
       'vpa_type', 'sellerid', 'bank_response', 'bank_name', 'pan_no_business',
       'pan_no_personal', 'gstin', 'gst_flag', 'category', 'sub_category',
       'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag', 'drr',
       'month_expected_gmv', 'total_child_mid', 'rank']]

len(master_format)

In [ ]:
gst_mapping = gst_data.set_index('solution_value')['solution_value.1'].to_dict()
master_format['GST Turn over'] = master_format['parent_id'].map(gst_mapping)

# Fill missing GSTs with 'NA'
master_format['GST Turn over'] = master_format['GST Turn over'].fillna('NA')

In [ ]:
ops_status_mapping = ops_status_data.set_index('bank_response')['final_remarks'].to_dict()
master_format['OPS team status'] = master_format['bank_response'].map(ops_status_mapping)

# Fill missing GSTs with 'NA'
master_format['OPS team status'] = master_format['OPS team status'].fillna('Pending for share')

In [ ]:
master_format.columns

In [ ]:

master_format['Proposed Limit'] = np.where(
    master_format['Month_expected_GMV'].isna(), 
    master_format['limit_flag'], 
    np.maximum(np.ceil(master_format['Month_expected_GMV'] / 1e5) * 1e5, master_format['limit_flag'])
)


In [ ]:

# First replace NaN in 'gmv' with 0 for calculation
master_format['% usage'] = (master_format['gmv'].fillna(0) / master_format['limit_flag']) * 100




In [ ]:

# Make sure % usage is numeric
master_format['% usage'] = pd.to_numeric(master_format['% usage'], errors='coerce')

# Define bins and labels
bins = [-float('inf'), 5, 15, 40, 60, 80, 100, float('inf')]
labels = ['1_<5 %', '2_5 to 15 %', '3_15-40%', '4_40-60%', '5_60-80%', '6_80-100%', '7.above 100%']

# Apply binning
master_format['Grouping'] = pd.cut(master_format['% usage'], bins=bins, labels=labels, right=False)



master_format['Grouping'].value_counts()


In [ ]:

# Suppose your DataFrame is named df
liquor_categories = ['Alcoholic Beverages', 'Restaurants and Bars', 'wine and liquor shop', 'Wine producers','Alcoholic beverages']

master_format['Liquor/Others'] = master_format['sub_category'].apply(lambda x: 'Liquor' if x in liquor_categories else 'Others')


master_format['Liquor/Others'].value_counts()



In [ ]:


master_format['Limit Required/Not'] = np.where(master_format['Proposed Limit'] > master_format['limit_flag'], 'Required', 'Not_required')

master_format['Limit Required/Not'].value_counts()


In [ ]:

master_format['Sum of GMV'] = master_format.groupby('bank_response')['gmv'].transform('sum')

master_format['Sum of DRR'] = master_format.groupby('bank_response')['DRR'].transform('sum')

master_format['Sum of Month Expected GMV'] = master_format.groupby('bank_response')['Month_expected_GMV'].transform('sum')

master_format['Sum of limit_flag'] = master_format.groupby('bank_response')['limit_flag'].transform('sum')

# Use numpy's vectorized operations  (MAX and ROUNDUP equivalent)
master_format['Sum of Proposed Limit'] = np.maximum(np.ceil(master_format['Sum of Month Expected GMV'] / 10**5) * 10**5, master_format['Sum of limit_flag'])

# Use numpy for the IF condition (AO2 > AN2)
master_format['Limit Required/Not.1'] = np.where(master_format['Sum of Proposed Limit'] > master_format['Sum of limit_flag'], "Required", "Not_required")

In [ ]:
master_format.head(5)

In [ ]:
master_format.to_csv('master_data.csv',index=False)